# Instituto Federal do Sudeste de Minas Gerais, Campus Barbacena

# Minicurso: Introdução a word embeddings



## 1. Word2vec - Mais exemplos


### 1.1. Game of Thrones

Alguns resultados de um treinamento onde utilizzei como dataset os 3 primeiros livros de Game of Thrones.

#### 1.1.1. Similaridade

A similaridade entre robb e stark é muito maior do que a similaridade entre tyrion e  stark, o que faz sentido visto que Robb é da família Stark enquanto Tyrion é um Lannister.
![](img/got1.png)

Exemplo de similaridade com palavras mais comuns.
![](img/got3.png)

As 10 palavras mais similares relacionadas aos sobrenomes Targaryen e Lannister.
![](img/got4.png)

### 1.1.2. Odd-One-Out

Odd-One-Out aplicado a personagens dos Livros. É interessante como o modelo consegue isolar personagens pertencentes a famílias diferentes.

![](img/got2.png)

### 1.2. Os Simpsons

Mais exemplos de resultados com o treinamento do wor2vec, dessa vez aplicado a um dataset com dados do desenho "Os Simpsons".

Fonte: https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

### 1.2.1. Similaridade

As 10 palavras mais similares a homer.
![](img/smp1.png)

As 10 palavras mais similares à bart.
![](img/smp2.png)

### 1.2.2. Odd-One-Out

Mais exemplos de Odd-One-Out isolando com sucesso personagens fora do contexto.
![](img/smp3.png)
![](img/smp4.png)
![](img/smp5.png)

### 1.1.3. Analogias

woman está para marge como homer está para...
![](img/smp6.png)
woman está para man como bart está para...
![](img/smp8.png)

## 2. Treinando seu Modelo

Hora de por em prática o que foi visto e treinar seu pŕoprio modelo.


### 2.1. Criando o Corpus

In [1]:
import numpy as np
import pandas as pd
from unidecode import unidecode
import nltk
import os
import re
from gensim.models import Word2Vec, KeyedVectors
from string import punctuation
#Pode ser alterado de acordo com a banda ou parcela do dataset escolhida. Existem musicas em diferentes linguas no dataset.
stopwords = nltk.corpus.stopwords.words('english')
#stopwords = nltk.corpus.stopwords.words('portuguese')
from nltk.stem import SnowballStemmer

In [2]:
pontuacao = list(punctuation)
# Função pra remover pontuação, stopwords e numeros de textos, pois essas são informações irrelevantes 
pontuacao.append('...')
pontuacao.append('\"')
pontuacao.append('\'')
pontuacao.append('``')
pontuacao.append('`')
pontuacao.append('\n')
pontuacao.append('\t')
pontuacao.append('\’')

def processa(txt):
    for p in pontuacao:
        txt = str(txt).replace(p, ' ')
    return ' '.join([t for t in txt.lower().split() 
                     if (len(t)>2) 
                     if not re.search('^\d', t) 
                     and not re.search('.*\d$', t) 
                     and (t not in stopwords)])

In [8]:
# Lendo os datasets
df1 = pd.read_csv('./music/lyrics1.csv')
df2 = pd.read_csv('./music/lyrics2.csv')
df3 = pd.read_csv('./music/lyrics3.csv')
df4 = pd.read_csv('./music/lyrics4.csv')
df5 = pd.read_csv('./music/lyrics5.csv')
df6 = pd.read_csv('./music/lyrics6.csv')
df7 = pd.read_csv('./music/lyrics7.csv')
df8 = pd.read_csv('./music/lyrics8.csv')
df9 = pd.read_csv('./music/lyrics9.csv')

In [9]:
# Concatenando os 2 datasets
df =  pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9])
df.head()

,Unnamed: 0,Band,Lyrics,Song
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds


In [10]:
# 516174 musicas disponíveis no dataset
df.shape

(516174, 4)

In [11]:
# 42627 Cantores/Bandas diferentes no dataset
len(df['Band'].unique())

42627

In [12]:
# Escolha uma ou mais bandas para treinar seu modelo
minhas_musicas = df[df['Band']=='Imagine Dragons']

In [13]:
# As 5 primeiras musicas da parcela escolhida
minhas_musicas.head()

,Unnamed: 0,Band,Lyrics,Song
41280,211280,Imagine Dragons,Ever since I could remember\r\nEverything insi...,Monster
41281,211281,Imagine Dragons,When the days are cold\nAnd the cards all fold...,Demons
41282,211282,Imagine Dragons,So this is what you meant\r\nWhen you said tha...,It's Time
41283,211283,Imagine Dragons,"I took a walk on a Saturday night,\nFog in the...",My Fault
41284,211284,Imagine Dragons,If you love somebody\r\nBetter tell them why t...,On Top of the World


In [14]:
# 60 musicad da banda escolhida. Escolha uma bonda com um numero consideravel de musicas
minhas_musicas.shape

(60, 4)

In [15]:
# Criando o corpus. Vale ressaltar que o corpus é uma lista de listas.
corpus = []

In [16]:
# Percorre linha or linha do dataset
for i in minhas_musicas.iterrows():
    corpus.append(processa(unidecode(str(i[1]['Lyrics']))).split(' '))

In [17]:
# Conferindo o corpus
corpus;

### 2.2. Treinando o Modelo

Experimente parâmetros diferentes, outras combinação de técnicas de pré-processamento, tente aprimorar os resultados. A dimensionalidade e o tamanho da janela são os que vão causa uma mudança mais expressiva. O motivo disso é testar, para perceber como os datasets reagem a cada configuração, e chegar a sua pŕopria conclusão de qual combinação de parâmetros e técnicas de pré-proessamento se adequa melhor à parela do daset escolhida, ou até mesmo a todo o dataset.


Outro experimento interessante é usar a mesma configuração pra treinar modelos com bandas diferentes e ver como palavras iguais são definidas em contextos diferentes para cada banda.

In [31]:
# Treinando o modelo 
from gensim.models import Word2Vec
model = Word2Vec(corpus, size=300, window=5, min_count=3, sg=0, iter=50)

In [32]:
model.train(corpus, total_examples=len(corpus), epochs=50)

(240699, 353800)

In [33]:
model.wv.most_similar(['love', 'lost'], topn=10)

[('real', 0.7515951991081238),
 ('dangerous', 0.7092431783676147),
 ('guess', 0.6717456579208374),
 ('trust', 0.6697754263877869),
 ('way', 0.661875307559967),
 ('feels', 0.6504908800125122),
 ('rings', 0.6215311288833618),
 ('something', 0.5959182977676392),
 ('headlights', 0.5866005420684814),
 ('try', 0.5499647855758667)]

In [34]:
model.wv.most_similar(['adrenaline'], topn=10)

[('takes', 0.9619840383529663),
 ('whatever', 0.9386924505233765),
 ('chains', 0.8262341022491455),
 ('veins', 0.7961289882659912),
 ('love', 0.6949001550674438),
 ('leaves', 0.6902086734771729),
 ('feels', 0.676640510559082),
 ('rip', 0.6728334426879883),
 ('cause', 0.6165151596069336),
 ('break', 0.6075426340103149)]

### 2.3. Words Movers Distance

Metodo utilizado para comparar setenças ou documentos.

In [35]:

#distancias.sort()
#distanciasdistancias= []
#l1 = corpus[0];
#    
#model.init_sims(replace=True)  # Normalizes the vectors in the word2vec class.
#for i in corpus:
#    distancias.append(model.wmdistance(l1, i)) 
#distancias.sort()
#distancias